<a href="https://colab.research.google.com/github/seonghunYang/big_contest2020/blob/master/EDA8_%EC%83%81%EB%8C%80%EC%A0%84%EC%A0%81%EB%B3%84_%ED%88%AC%EC%88%98_%EC%84%B8%EC%9D%B4%EB%B2%84%EB%A9%94%ED%8A%B8%EB%A6%AD%EC%8A%A4_%EC%83%9D%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
path = './gdrive/My Drive/빅콘테스트2020/데이터/2020빅콘테스트 데이터분석분야-퓨쳐스리그_스포츠투아이_제공데이터(.CSV)_시즌별, 시트별 구분/'

In [2]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
def load_Data(name):
    years = ['2016','2017','2018','2019','2020']
    name_salt = '2020빅콘테스트_스포츠투아이_제공데이터_'
    PATH = path
    df = pd.DataFrame()
    for year in years:
        if year == '2016':
            df = pd.read_csv(PATH + name_salt + name+'_'+year+'.csv', encoding='CP949')
        else :
            new_df = pd.read_csv(PATH + name_salt + name+'_'+year+'.csv', encoding='CP949')
            df = pd.concat([df, new_df])
    return df

In [4]:
team_pitcher = load_Data('팀투수')
pitcher = load_Data('개인투수')

In [ ]:
team_pitcher

,G_ID,GDAY_DS,T_ID,VS_T_ID,HEADER_NO,TB_SC,CG_CK,WLS,HOLD,INN2,BF,PA,AB,HIT,H2,H3,HR,SB,CS,SH,SF,BB,IB,HP,KK,GD,WP,BK,ERR,R,ER,P_WHIP_RT,P2_WHIP_RT,CB_WHIP_RT
0,20160401HHLG0,20160401,LG,HH,0,B,0,W,0,36,184,52,46,13,2,0,0,0,0,3,0,3,0,0,10,1,0,0,0,4,4,0.642857,1.285714,2.400000
1,20160401HHLG0,20160401,HH,LG,0,T,0,L,0,34,196,47,42,9,2,0,1,2,1,1,0,4,0,0,11,0,1,0,0,5,4,1.500000,1.000000,0.750000
2,20160401HTNC0,20160401,NC,HT,0,B,0,W,0,27,143,38,34,8,3,0,1,0,0,0,0,3,0,1,10,1,2,0,0,4,4,1.333333,1.038462,2.142857
3,20160401HTNC0,20160401,HT,NC,0,T,0,L,0,24,147,36,30,9,2,0,2,0,0,1,0,5,0,0,9,1,0,0,0,5,5,0.500000,1.695652,1.875000
4,20160401KTSK0,20160401,SK,KT,0,B,0,L,0,27,151,42,36,11,0,0,2,2,1,0,1,4,0,1,7,0,0,0,0,8,8,1.000000,2.357143,2.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,20200719LTSS0,20200719,LT,SS,0,T,0,W,2,27,155,34,31,7,0,0,1,2,1,0,0,3,0,0,9,1,0,0,0,1,1,0.600000,1.111111,1.500000
636,20200719OBHT0,20200719,HT,OB,0,B,0,L,0,27,137,41,37,11,3,0,1,2,1,0,0,2,1,2,4,0,0,0,0,8,8,2.142857,1.800000,0.750000
637,20200719OBHT0,20200719,OB,HT,0,T,0,W,1,27,177,38,32,8,0,1,0,0,0,0,0,6,0,0,5,1,0,0,0,4,4,3.000000,1.000000,1.875000
638,20200719WOSK0,20200719,SK,WO,0,B,0,W,0,27,134,37,33,7,0,0,1,1,0,0,0,3,0,1,4,1,0,0,0,3,3,1.500000,1.105263,2.142857


In [ ]:
new_columns = "게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,완투,결과,홀드,이닝*3,투구수,타자,타수,안타,2루타,3루타,홈런,도루,도루실패,희타,희비,4구,고4,사구,삼진,병살타,폭투,보크,실책,실점,자책점,득점권WHIP,2점차이하WHIP,345번타자WHIP"
new_columns = new_columns.split(',')

In [ ]:
team_pitcher.columns = new_columns

In [ ]:
team_pitcher['일자'] = pd.to_datetime(team_pitcher['일자'],format='%Y%m%d', errors='raise')
team_pitcher['시즌'] = team_pitcher['일자'].dt.year 

In [ ]:
C_dic = {}
for year in range(2016,2021):
  team_pitcher_year = team_pitcher[team_pitcher['시즌'] == year].copy()
  team_pitcher_year = team_pitcher_year.sum() 
  C = (( 9 * team_pitcher_year['자책점'] - ( 13 * team_pitcher_year['홈런'] + 3 * (team_pitcher_year['4구'] + team_pitcher_year['사구'] - team_pitcher_year['고4']) - 2 * team_pitcher_year['삼진'])) / (team_pitcher_year['이닝*3'] / 3))
  C_dic[year] = C

In [ ]:
def createERA(row):
    previous = team_pitcher[((team_pitcher['팀코드'] == row['팀코드']) & (team_pitcher['상대팀코드'] == row['상대팀코드'])) & (team_pitcher['상대팀코드'] == row['상대팀코드'])].copy()
    previous = previous[(previous['시즌'] == row['시즌']) &(previous['일자'] < row['일자'])]
    if len(previous) == 0:
      row['평균자책점'] = 0
      return row
    up = previous['자책점'].sum()
    down = previous['이닝*3'].sum() / 3
    if down == 0:
        row['평균자책점']
        return row
    row['평균자책점'] = (up * 9) / down
    return row

def createRA(row):
    previous = team_pitcher[(team_pitcher['팀코드'] == row['팀코드']) & (team_pitcher['상대팀코드'] == row['상대팀코드'])].copy()
    previous = previous[(previous['시즌'] == row['시즌']) &(previous['일자'] < row['일자'])]
    if len(previous) == 0:
      row['RA/9'] = 0
      return row
    up = previous['실점'].sum()
    down = previous['이닝*3'].sum() / 3
    if down == 0:
        row['RA/9']
        return row
    row['RA/9'] = (up * 9) / down
    return row

def createERC(row):
    previous = team_pitcher[(team_pitcher['팀코드'] == row['팀코드']) & (team_pitcher['상대팀코드'] == row['상대팀코드'])].copy()
    previous = previous[(previous['시즌'] == row['시즌']) &(previous['일자'] < row['일자'])]
    if len(previous) == 0:
      row['ERC'] = 0
      return row
    previous_sum = previous.sum()
    PTB = 0.89 * (1.255 * (previous_sum['안타'] - previous_sum['홈런']) + 4 * previous_sum['홈런']) + 0.475 * (previous_sum['4구'] + previous_sum['사구'] - previous_sum['고4'])
    ERC = 9* ( ((previous_sum['안타'] + previous_sum['4구'] + previous_sum['사구']) * PTB) / (previous_sum['타자'] * previous_sum['이닝*3'] / 3) ) - 0.56
    if ERC < 2.24:
      ERC = 9* ( ((previous_sum['안타'] + previous_sum['4구'] + previous_sum['사구']) * PTB) / (previous_sum['타자'] * previous_sum['이닝*3'] / 3) ) * 0.75
    row['ERC'] = ERC
    return row

def createFIP(row):
    previous = team_pitcher[(team_pitcher['시즌'] == row['시즌']) & (team_pitcher['팀코드'] == row['팀코드']) & (team_pitcher['상대팀코드'] == row['상대팀코드']) & (team_pitcher['일자'] < row['일자'])]
    if len(previous) == 0:
        row['FIP'] = 0
        return row
    previous_sum = previous.sum()
    FIP = (( 13 * previous_sum['홈런'] + 3 * (previous_sum['4구'] + previous_sum['사구'] - previous_sum['고4']) - 2 * previous_sum['삼진']) / (previous_sum['이닝*3'] / 3)) + C_dic[row['시즌']]
    row['FIP'] = FIP
    return row

def createLOB(row):
    previous = team_pitcher[(team_pitcher['시즌'] == row['시즌']) & (team_pitcher['팀코드'] == row['팀코드']) & (team_pitcher['상대팀코드'] == row['상대팀코드']) & (team_pitcher['일자'] < row['일자'])]
    if len(previous) == 0:
        row['LOB'] = 0
        return row
    previous_sum = previous.sum()
    up = previous_sum['안타'] + previous_sum['4구'] + previous_sum['사구'] - previous_sum['실점']
    down = previous_sum['안타'] + previous_sum['4구'] + previous_sum['사구'] - (1.4 * previous_sum['홈런'])
    if down == 0:
      row['LOB'] = 0
      return row 
    row['LOB'] = up / down
    return row

def createK_HR(row):
    previous = team_pitcher[(team_pitcher['시즌'] == row['시즌']) & (team_pitcher['팀코드'] == row['팀코드']) & (team_pitcher['상대팀코드'] == row['상대팀코드']) & (team_pitcher['일자'] < row['일자'])]
    if len(previous) == 0:
        row['홈런율'] = 0
        row['삼진율'] = 0
        return row
    previous_sum = previous.sum()
    up_HR = previous_sum['홈런'] 
    up_k = previous_sum['삼진']
    down = previous_sum['타자'] - previous_sum['고4']
    if down == 0:
        row['홈런율'] = 0
        row['삼진율'] = 0
        return row
    row['홈런율'] = up_HR / down
    row['삼진율'] = up_k / down
    return row

def create9(row):
    previous = team_pitcher[(team_pitcher['시즌'] == row['시즌']) & (team_pitcher['팀코드'] == row['팀코드']) & (team_pitcher['상대팀코드'] == row['상대팀코드']) & (team_pitcher['일자'] < row['일자'])]
    if len(previous) == 0:
        row['K9'] = 0
        row['BB9'] = 0
        row['HR9'] = 0
        row['KBB'] = 0
        return row
    previous_sum = previous.sum()
    up_K = previous_sum['삼진'] 
    up_BB = previous_sum['4구'] - previous_sum['고4']
    up_HR = previous_sum['홈런']
    down = previous_sum['이닝*3'] / 3
    if down == 0 or up_BB == 0 :
        row['K9'] = 0
        row['BB9'] = 0
        row['HR9'] = 0
        row['KBB'] = 0
        return row
    row['K9'] = up_K / down * 9
    row['BB9'] = up_BB / down * 9
    row['HR9'] = up_HR / down * 9
    row['KBB'] = up_K / up_BB
    return row

In [ ]:
try:
  team_pitcher = team_pitcher.apply(createERA, axis=1)
  team_pitcher = team_pitcher.apply(createRA, axis=1)
  team_pitcher = team_pitcher.apply(createERC, axis=1)
  team_pitcher = team_pitcher.apply(createFIP, axis=1)
  team_pitcher = team_pitcher.apply(createLOB, axis=1)
  team_pitcher = team_pitcher.apply(createK_HR, axis=1)
  team_pitcher = team_pitcher.apply(create9, axis=1)
except:
  team_pitcher.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA8-상대전적_투수_세이버 매트릭스_합본.csv', encoding='utf-8-sig', index=False)
finally:
  team_pitcher.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA8-상대전적_투수_세이버 매트릭스_합본.csv', encoding='utf-8-sig', index=False)

In [ ]:
team_pitcher

,게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,완투,결과,홀드,이닝*3,투구수,타자,타수,안타,2루타,3루타,홈런,도루,도루실패,희타,희비,4구,고4,사구,삼진,병살타,폭투,보크,실책,실점,자책점,득점권WHIP,2점차이하WHIP,345번타자WHIP,시즌,평균자책점,RA/9,ERC,FIP,LOB,홈런율,삼진율,K9,BB9,HR9,KBB
0,20160401HHLG0,2016-04-01,LG,HH,0,B,0,W,0,36,184,52,46,13,2,0,0,0,0,3,0,3,0,0,10,1,0,0,0,4,4,0.642857,1.285714,2.400000,2016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,20160401HHLG0,2016-04-01,HH,LG,0,T,0,L,0,34,196,47,42,9,2,0,1,2,1,1,0,4,0,0,11,0,1,0,0,5,4,1.500000,1.000000,0.750000,2016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,20160401HTNC0,2016-04-01,NC,HT,0,B,0,W,0,27,143,38,34,8,3,0,1,0,0,0,0,3,0,1,10,1,2,0,0,4,4,1.333333,1.038462,2.142857,2016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,20160401HTNC0,2016-04-01,HT,NC,0,T,0,L,0,24,147,36,30,9,2,0,2,0,0,1,0,5,0,0,9,1,0,0,0,5,5,0.500000,1.695652,1.875000,2016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,20160401KTSK0,2016-04-01,SK,KT,0,B,0,L,0,27,151,42,36,11,0,0,2,2,1,0,1,4,0,1,7,0,0,0,0,8,8,1.000000,2.357143,2.250000,2016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,20200719LTSS0,2020-07-19,LT,SS,0,T,0,W,2,27,155,34,31,7,0,0,1,2,1,0,0,3,0,0,9,1,0,0,0,1,1,0.600000,1.111111,1.500000,2020,6.211268,6.211268,5.269202,5.498131,0.647969,0.027778,0.138889,5.704225,3.929577,1.140845,1.451613
636,20200719OBHT0,2020-07-19,HT,OB,0,B,0,L,0,27,137,41,37,11,3,0,1,2,1,0,0,2,1,2,4,0,0,0,0,8,8,2.142857,1.800000,0.750000,2020,4.434783,5.608696,4.985167,4.031506,0.645161,0.012945,0.152104,6.130435,2.347826,0.521739,2.611111
637,20200719OBHT0,2020-07-19,OB,HT,0,T,0,W,1,27,177,38,32,8,0,1,0,0,0,0,0,6,0,0,5,1,0,0,0,4,4,3.000000,1.000000,1.875000,2020,2.957143,3.985714,3.882761,4.368152,0.736515,0.013158,0.144737,5.657143,3.728571,0.514286,1.517241
638,20200719WOSK0,2020-07-19,SK,WO,0,B,0,W,0,27,134,37,33,7,0,0,1,1,0,0,0,3,0,1,4,1,0,0,0,3,3,1.500000,1.105263,2.142857,2020,6.717073,7.112195,6.390847,5.733308,0.646018,0.030675,0.196319,8.429268,5.531707,1.317073,1.523810


In [5]:
new_columns = "게임키 일자 팀코드 상대팀코드 더블헤더코드 초말 선수코드 선발 구원 완투 종료 결과 홀드 이닝*3 투구수 타자 타수 안타 2루타 3루타 홈런 도루 도루 희타 희비 4구 고4 사구 삼진 병살 폭투 보크 실책 실점 자책점 득점권WHIP 2점차이하WHIP 345번타자WHIP"
new_columns = new_columns.split(' ')

In [6]:
pitcher.columns = new_columns 

In [7]:
pitcher['일자'] = pd.to_datetime(pitcher['일자'],format='%Y%m%d', errors='raise')
pitcher['시즌'] = pitcher['일자'].dt.year 

In [8]:
C_dic = {2016: 3.7732111715557406,
 2017: 3.7201314451138408,
 2018: 3.810512298292339,
 2019: 3.3880686560934894,
 2020: 3.596723044397463}

In [9]:
def createERA(row):
    previous = pitcher[(pitcher['선수코드'] == row['선수코드']) & (pitcher['상대팀코드'] == row['상대팀코드'])].copy()
    previous = previous[(previous['시즌'] == row['시즌']) &(previous['일자'] < row['일자'])]
    if len(previous) == 0:
      row['평균자책점'] = 0
      return row
    up = previous['자책점'].sum()
    down = previous['이닝*3'].sum() / 3
    if down == 0:
        row['평균자책점'] = 0
        return row
    row['평균자책점'] = (up * 9) / down
    return row

def createRA(row):
    previous = pitcher[(pitcher['선수코드'] == row['선수코드']) & (pitcher['상대팀코드'] == row['상대팀코드'])].copy()
    previous = previous[(previous['시즌'] == row['시즌']) &(previous['일자'] < row['일자'])]
    if len(previous) == 0:
      row['RA/9'] = 0
      return row
    up = previous['실점'].sum()
    down = previous['이닝*3'].sum() / 3
    if down == 0:
        row['RA/9'] = 0
        return row
    row['RA/9'] = (up * 9) / down
    return row

def createERC(row):
    previous = pitcher[(pitcher['선수코드'] == row['선수코드']) & (pitcher['상대팀코드'] == row['상대팀코드'])].copy()
    previous = previous[(previous['시즌'] == row['시즌']) &(previous['일자'] < row['일자'])]
    if len(previous) == 0:
      row['ERC'] = 0
      return row
    previous_sum = previous.sum()
    PTB = 0.89 * (1.255 * (previous_sum['안타'] - previous_sum['홈런']) + 4 * previous_sum['홈런']) + 0.475 * (previous_sum['4구'] + previous_sum['사구'] - previous_sum['고4'])
    down = (previous_sum['타자'] * previous_sum['이닝*3'] / 3)
    if down == 0:
      row['ERC'] = 0
      return row
    ERC = 9* ( ((previous_sum['안타'] + previous_sum['4구'] + previous_sum['사구']) * PTB) / down ) - 0.56
    if ERC < 2.24:
      ERC = 9* ( ((previous_sum['안타'] + previous_sum['4구'] + previous_sum['사구']) * PTB) / down ) * 0.75
    row['ERC'] = ERC
    return row

def createFIP(row):
    previous = pitcher[(pitcher['시즌'] == row['시즌']) & (pitcher['선수코드'] == row['선수코드']) & (pitcher['일자'] < row['일자']) & (pitcher['상대팀코드'] == row['상대팀코드'])]
    if len(previous) == 0:
        row['FIP'] = 0
        return row
    previous_sum = previous.sum()
    down = (previous_sum['이닝*3'] / 3)
    if down == 0 :
      row['FIP'] = 0
      return row
    FIP = (( 13 * previous_sum['홈런'] + 3 * (previous_sum['4구'] + previous_sum['사구'] - previous_sum['고4']) - 2 * previous_sum['삼진']) /down) + C_dic[row['시즌']]
    row['FIP'] = FIP
    return row

def createLOB(row):
    previous = pitcher[(pitcher['시즌'] == row['시즌']) & (pitcher['선수코드'] == row['선수코드']) & (pitcher['일자'] < row['일자']) & (pitcher['상대팀코드'] == row['상대팀코드'])]
    if len(previous) == 0:
        row['LOB'] = 0
        return row
    previous_sum = previous.sum()
    up = previous_sum['안타'] + previous_sum['4구'] + previous_sum['사구'] - previous_sum['실점']
    down = previous_sum['안타'] + previous_sum['4구'] + previous_sum['사구'] - (1.4 * previous_sum['홈런'])
    if down == 0:
      row['LOB'] = 0
      return row 
    row['LOB'] = up / down
    return row

def createK_HR(row):
    previous = pitcher[(pitcher['시즌'] == row['시즌']) & (pitcher['선수코드'] == row['선수코드']) & (pitcher['일자'] < row['일자']) & (pitcher['상대팀코드'] == row['상대팀코드'])]
    if len(previous) == 0:
        row['홈런율'] = 0
        row['삼진율'] = 0
        return row
    previous_sum = previous.sum()
    up_HR = previous_sum['홈런'] 
    up_k = previous_sum['삼진']
    down = previous_sum['타자'] - previous_sum['고4']
    if down == 0:
        row['홈런율'] = 0
        row['삼진율'] = 0
        return row
    row['홈런율'] = up_HR / down
    row['삼진율'] = up_k / down
    return row

def create9(row):
    previous = pitcher[(pitcher['시즌'] == row['시즌']) & (pitcher['선수코드'] == row['선수코드']) & (pitcher['일자'] < row['일자']) & (pitcher['상대팀코드'] == row['상대팀코드'])]
    if len(previous) == 0:
        row['K9'] = 0
        row['BB9'] = 0
        row['HR9'] = 0
        row['KBB'] = 0
        return row
    previous_sum = previous.sum()
    up_K = previous_sum['삼진'] 
    up_BB = previous_sum['4구'] - previous_sum['고4']
    up_HR = previous_sum['홈런']
    down = previous_sum['이닝*3'] / 3
    if down == 0 or up_BB == 0 :
        row['K9'] = 0
        row['BB9'] = 0
        row['HR9'] = 0
        row['KBB'] = 0
        return row
    row['K9'] = up_K / down * 9
    row['BB9'] = up_BB / down * 9
    row['HR9'] = up_HR / down * 9
    row['KBB'] = up_K / up_BB
    return row



In [10]:
try:
  pitcher = pitcher.apply(createERA, axis=1)
  pitcher = pitcher.apply(createRA, axis=1)
  pitcher = pitcher.apply(createERC, axis=1)
except:
  pitcher.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA8-상대전적_개인투수_세이버 매트릭스_합본.csv', encoding='utf-8-sig', index=False)
finally:
  pitcher.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA8-상대전적_개인투수_세이버 매트릭스_합본.csv', encoding='utf-8-sig', index=False)

In [11]:
try:
  pitcher = pitcher.apply(createFIP, axis=1)
  pitcher = pitcher.apply(createLOB, axis=1)
  pitcher = pitcher.apply(createK_HR, axis=1)
  pitcher = pitcher.apply(create9, axis=1)
except:
  pitcher.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA8-상대전적_개인투수_세이버 매트릭스_합본.csv', encoding='utf-8-sig', index=False)
finally:
  pitcher.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA8-상대전적_개안투수_세이버 매트릭스_합본.csv', encoding='utf-8-sig', index=False)